In [5]:
from PIL import Image
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import glacierml as gl
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.cluster import KMeans
import seaborn as sns
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_columns', None)



        
# RGI.to_csv('Farinotti_mean_thickness_RGI_ID.csv')


In [6]:
df1= pd.read_csv('reference_thicknesses/Farinotti_vol.csv')
df1 = df1.drop('Unnamed: 0', axis = 1)
df1 = df1.dropna()

RGI = gl.RGI_loader(
    pth = '/home/simonhans/data/prethicktor/RGI/rgi60-attribs/'
)
RGI = RGI.sort_values('RGIId')

RGIf = pd.merge(RGI, df1, on = 'RGIId')
RGIf = RGIf.dropna()

df = RGIf
df = df.dropna()
df['region'] = df['RGIId'].str[6:8]
df = df.rename(columns = {'Area_x':'Area'})

for region_number in (range(1,20,1)):
    if len(str(region_number)) == 1:
        N = 1
        region_number = str(region_number).zfill(N + len(str(region_number)))
    else:
        region_number == str(region_number)
        

    if region_number != 19:
        drops = df[
            ((df['region'] == str(region_number)) & (df['Zmin'] < 0)) |
            ((df['region'] == str(region_number)) & (df['Zmed'] < 0)) |
            ((df['region'] == str(region_number)) & (df['Zmax'] < 0)) |
            ((df['region'] == str(region_number)) & (df['Slope'] < 0)) |
            ((df['region'] == str(region_number)) & (df['Aspect'] < 0))
        ].index
        
        if not drops.empty:
            df = df.drop(drops)

# print('df length before dropping bad data: ' + str(len(df)))
# len_1 = len(df)
# df = df.drop(df.loc[df['Zmed']<0].index)
# df = df.drop(df.loc[df['Lmax']<0].index)
# df = df.drop(df.loc[df['Slope']<0].index)
# df = df.drop(df.loc[df['Zmin']<0].index)
# df = df.drop(df.loc[df['Zmax']<0].index)
# print('df length after dropping bad data: ' + str(len(df)))
# df = df.reset_index()
# df = df.drop('index', axis = 1)
# len_2 = len(df)
# print(len_1 - len_2)
kmeans = KMeans(n_clusters = 2, random_state = 0)
dft = df[[
    # 'CenLat',
    # 'CenLon',
    # 'Area',
    # 'Aspect',
    'Lmax',
    # 'Slope',
    'Zmin',
    # 'Zmax'
]]
df['cluster'] = kmeans.fit_predict(dft)
df['region'] = df['RGIId'].str[6:8]

df = df.rename(columns = {
    '0':'Farinotti Volume',
    'Area_y':'Farinotti Area'
})
df['Farinotti Thickness'] = df['Farinotti Volume'] / (df['Farinotti Area'] * 1e6)





In [7]:
df = df.reset_index()
df = df.drop('index', axis = 1)
df

,RGIId,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,Farinotti Volume,Farinotti Area,region,cluster,Farinotti Thickness
0,RGI60-01.00001,63.6890,-146.8230,42.0,1936,2385,2725,0.360,346,839,7.638771e+06,0.360625,01,0,21.182034
1,RGI60-01.00002,63.4040,-146.6680,16.0,1713,2005,2144,0.558,162,1197,1.697646e+07,0.556250,01,0,30.519479
2,RGI60-01.00003,63.3760,-146.0800,18.0,1609,1868,2182,1.685,175,2106,5.969346e+07,1.680625,01,0,35.518605
3,RGI60-01.00004,63.3810,-146.1200,19.0,1273,1944,2317,3.681,195,4175,1.952248e+08,3.678125,01,0,53.077261
4,RGI60-01.00005,63.5510,-147.0570,16.0,1494,1914,2317,2.573,181,2981,1.221541e+08,2.570625,01,0,47.519206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215490,RGI60-19.02748,-53.9860,-37.7325,29.9,310,-999,510,0.042,315,255,5.502906e+05,0.042500,19,0,12.948013
215491,RGI60-19.02749,-54.8310,-36.1361,23.6,330,-999,830,0.567,200,1130,1.300672e+07,0.568125,19,0,22.894110
215492,RGI60-19.02750,-54.1884,-37.3018,16.8,10,-999,1110,4.118,308,4329,2.506893e+08,4.118125,19,0,60.874612
215493,RGI60-19.02751,-68.8656,-90.4266,0.4,170,-999,270,0.011,122,106,1.068206e+05,0.010625,19,0,10.053704


In [ ]:
df

In [4]:
arch = '10-5'
top_learning_rate = 0.01
epochs = 100
dropout = 1

df7 = gl.data_loader(
    RGI_input = 'y',
    scale = 'g',
    area_scrubber = 'off',
#     anomaly_input = 5
root_dir = '/home/simonhans/data/prethicktor/'
)
# df5 = df5.drop('RGIId', axis = 1)
# df5 = df5.drop('Zmed', axis = 1)
module = 'sm7'
res = 'sr7'
dataset = df7
dataset.name = 'df7'
rootdir = 'saved_models/' + module + '/sm_' + str(arch) + '/'
print('df length = ' + str(len(df)))
RGI_for_predictions = df[[
    'CenLat',
    'CenLon',
    'Slope',
    'Zmin',
    'Zmed',
    'Zmax',
    'Area',
    'Aspect',
    'Lmax',
]]
print('RGI length = ' + str(len(RGI_for_predictions)))
dnn_model = {}
RS = range(0,25,1)
df_y = pd.DataFrame()
for rs in tqdm((RS)):
    model_name = (
        str(arch) + 
        '_' + 
        dataset.name +
        '_' + 
        str(dropout) +
        '_dnn_MULTI_' + 
        str(top_learning_rate) + 
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )   

    model_path = (
        rootdir + 
        dataset.name +
        '_' + 
        str(dropout) +
        '_dnn_MULTI_' + 
        str(top_learning_rate) + 
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )

    dnn_model[model_name] = tf.keras.models.load_model(model_path)


    y = pd.DataFrame(dnn_model[model_name].predict(RGI_for_predictions, verbose = 0))
    df_y = pd.concat([df_y,y], axis = 1)
#     print('y length = ' + str(len(y))) 1)

df length = 215495
RGI length = 215495


  0%|                                                    | 0/25 [00:00<?, ?it/s]2022-08-18 14:42:28.955858: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-18 14:42:28.955884: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-18 14:42:28.955907: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sbook): /proc/driver/nvidia/version does not exist
2022-08-18 14:42:28.956104: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
  0%|                                

KeyboardInterrupt: 

In [16]:
df['Edasi Thickness'] = df_y.mean(axis = 1)
df['VE - VF'] = df['Edasi Thickness'] - df['Farinotti Thickness']
df['VE / VF'] = df['Edasi Thickness'] / df['Farinotti Thickness']

In [17]:
df_y

,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,67.423920,82.132080,71.008865,89.096916,80.597092,67.494225,72.940933,86.537712,75.938698,80.711845,77.135895,58.230385,72.472130,72.397530,80.413200,78.726273,61.017780,79.465942,77.100647,71.820053,67.903511,76.124626,63.981499,79.160454,78.246872
1,80.973389,91.294968,86.403076,98.297913,106.049492,89.433266,88.120148,91.993279,84.507004,88.532875,91.590492,68.351189,89.184074,84.744751,93.876289,85.274704,77.692154,93.380959,88.604462,88.516670,78.207008,89.316666,78.620438,91.640038,85.394569
2,79.512344,88.158524,85.464073,95.741310,101.190193,87.020622,85.732246,88.600471,82.219681,87.130669,88.464943,67.439537,86.178482,81.545189,90.603294,83.673943,75.636765,90.052948,84.282974,85.331413,75.831200,87.244148,76.714401,88.428886,83.708832
3,82.662804,87.573143,90.819725,100.193413,98.733803,87.159386,87.097939,89.289253,86.349030,91.983284,88.652321,73.040916,87.763161,81.844955,90.859642,85.491249,75.831604,91.257011,80.759903,85.098167,77.764626,89.511528,78.756042,89.295097,89.861656
4,81.488045,88.555588,88.173866,98.369064,100.423386,87.467857,86.857399,89.814415,84.940643,89.782265,88.829765,70.138519,87.276344,82.412857,91.790123,85.211891,75.947952,91.091583,83.137527,84.841164,77.278580,89.228401,78.113029,89.431007,86.938232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215490,74.513847,69.951096,115.667015,101.942192,95.405998,98.081833,91.982910,60.317120,72.764847,87.209457,90.322403,90.799286,93.833344,101.045486,65.601929,69.846336,82.920990,94.552704,118.874695,96.711655,105.489281,84.563225,56.154995,52.716660,66.557327
215491,84.512413,80.724365,126.834724,108.729134,104.447105,110.986252,100.792130,69.726334,80.632988,96.068214,95.071945,99.359489,103.687462,118.077713,65.367126,78.310349,90.633362,102.878418,126.597626,104.408607,110.288605,91.981651,75.509201,63.961224,79.670273
215492,80.038170,70.389389,111.252892,103.720482,97.170830,97.760551,97.425858,67.876358,74.218887,85.723457,89.866341,93.696404,95.452232,107.220222,69.361038,75.336533,83.499077,92.002502,112.691055,92.668213,110.972008,87.803261,73.022896,64.077461,84.559433
215493,101.157631,96.902016,146.837341,123.982841,114.300072,129.059311,116.459557,86.114540,97.780090,110.189552,103.969231,112.838875,118.531898,133.451202,66.041718,89.565300,104.071579,118.481384,132.610840,128.044098,123.628136,105.499992,80.806274,83.258072,92.314690


In [18]:
df

,RGIId,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,Farinotti Volume,Farinotti Area,region,cluster,Farinotti Thickness,Edasi Thickness,VE - VF,VE / VF
0,RGI60-01.00001,63.6890,-146.8230,42.0,1936,2385,2725,0.360,346,839,7.638771e+06,0.360625,01,0,21.182034,74.723167,53.541133,3.527667
1,RGI60-01.00002,63.4040,-146.6680,16.0,1713,2005,2144,0.558,162,1197,1.697646e+07,0.556250,01,0,30.519479,87.599998,57.080520,2.870298
2,RGI60-01.00003,63.3760,-146.0800,18.0,1609,1868,2182,1.685,175,2106,5.969346e+07,1.680625,01,0,35.518605,85.036270,49.517665,2.394133
3,RGI60-01.00004,63.3810,-146.1200,19.0,1273,1944,2317,3.681,195,4175,1.952248e+08,3.678125,01,0,53.077261,86.705986,33.628725,1.633581
4,RGI60-01.00005,63.5510,-147.0570,16.0,1494,1914,2317,2.573,181,2981,1.221541e+08,2.570625,01,0,47.519206,86.301582,38.782376,1.816141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215490,RGI60-19.02748,-53.9860,-37.7325,29.9,310,-999,510,0.042,315,255,5.502906e+05,0.042500,19,0,12.948013,85.513069,72.565056,6.604339
215491,RGI60-19.02749,-54.8310,-36.1361,23.6,330,-999,830,0.567,200,1130,1.300672e+07,0.568125,19,0,22.894110,94.770271,71.876162,4.139505
215492,RGI60-19.02750,-54.1884,-37.3018,16.8,10,-999,1110,4.118,308,4329,2.506893e+08,4.118125,19,0,60.874612,88.712219,27.837608,1.457294
215493,RGI60-19.02751,-68.8656,-90.4266,0.4,170,-999,270,0.011,122,106,1.068206e+05,0.010625,19,0,10.053704,108.635857,98.582152,10.805555


In [19]:
for region_number in tqdm(range(1,20,1)):
    if len(str(region_number)) == 1:
        N = 1
        region_number = str(region_number).zfill(N + len(str(region_number)))
    else:
        str(region_number) == str(region_number)
        
    dft = df[df['region'] == str(region_number)]

    RGI_for_predictions = dft[[
        'CenLat',
        'CenLon',
        'Slope',
        'Zmin',
        'Zmed',
        'Zmax',
        'Area',
        'Aspect',
        'Lmax',
    ]]

    fig = plt.figure(figsize=(25, 25))
    plt.subplots_adjust(hspace=0.5)
    plt.suptitle(
        'Volume estimates using layer architecture ' + str(arch) +
        ', learning rate: ' + str(top_learning_rate) + ', epochs: ' + str(epochs) +
        '\n for region ' + str(region_number)
                 , fontsize=24, y=0.98
                )
    fig.patch.set_facecolor('w')

    dnn_model = {}
    RS = range(0,25,1)
    for n, rs in (enumerate(RS)):
        ax = plt.subplot(5, 5, n + 1)
        model_name = (
            str(arch) + 
            '_' + 
            dataset.name +
            '_' + 
            str(dropout) +
            '_dnn_MULTI_' + 
            str(top_learning_rate) + 
            '_0.2_' +
            str(epochs) + 
            '_' + 
            str(rs)
        )   

        model_path = (
            rootdir + 
            dataset.name +
            '_' + 
            str(dropout) +
            '_dnn_MULTI_' + 
            str(top_learning_rate) + 
            '_0.2_' +
            str(epochs) + 
            '_' + 
            str(rs)
        )

        dnn_model[model_name] = tf.keras.models.load_model(model_path)


        y = dnn_model[model_name].predict(RGI_for_predictions, verbose = 0)
        # df = pd.concat([df, pd.Series(y.flatten(), name = rs)], axis = 1)

        plt.scatter(
            x = dft['Farinotti Thickness'],
            y = y,
            marker = '.',
            c = dft['cluster'],
            cmap = 'viridis',
#             alpha = 0.5
#             sharex = True,
#             sahrey = True
        )
        plt.plot(
            (0,(dft['Farinotti Thickness'].max())),
            (0,(dft['Farinotti Thickness'].max())),
            '-',
            c = 'orange'

        )
        plt.colorbar()
        plt.xlabel('Farinotti Thickness (m)')
        plt.ylabel('Edasi Thickness (m)')
        ax.set_title('Random State ' +str(rs))
        plt.yscale('log')
        plt.xscale('log')
#     break
#         plt.xlim((0,400))
#         plt.ylim((0,400))
#         plt.savefig(
#             'figs/regional_crossplots/' + str(arch) + '_' +
#             str(top_learning_rate) + '_' +
#             str(epochs) + '_' +
#             str(dropout) + '_' + str(region_number) + '_cluster.eps'
#                    )

#         plt.savefig(
#             'figs/regional_crossplots/' + str(arch) + '_' +
#             str(top_learning_rate) + '_' +
#             str(epochs) + '_' +
#             str(dropout) + '_' + str(region_number) + '_cluster.png'
#        )


100%|███████████████████████████████████████████| 19/19 [04:50<00:00, 15.28s/it]


Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fd6d031b680> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fd6d031b200> (for post_execute):


KeyboardInterrupt: 

In [20]:
df

,RGIId,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,Farinotti Volume,Farinotti Area,region,cluster,Farinotti Thickness,Edasi Thickness,VE - VF,VE / VF
0,RGI60-01.00001,63.6890,-146.8230,42.0,1936,2385,2725,0.360,346,839,7.638771e+06,0.360625,01,0,21.182034,74.723167,53.541133,3.527667
1,RGI60-01.00002,63.4040,-146.6680,16.0,1713,2005,2144,0.558,162,1197,1.697646e+07,0.556250,01,0,30.519479,87.599998,57.080520,2.870298
2,RGI60-01.00003,63.3760,-146.0800,18.0,1609,1868,2182,1.685,175,2106,5.969346e+07,1.680625,01,0,35.518605,85.036270,49.517665,2.394133
3,RGI60-01.00004,63.3810,-146.1200,19.0,1273,1944,2317,3.681,195,4175,1.952248e+08,3.678125,01,0,53.077261,86.705986,33.628725,1.633581
4,RGI60-01.00005,63.5510,-147.0570,16.0,1494,1914,2317,2.573,181,2981,1.221541e+08,2.570625,01,0,47.519206,86.301582,38.782376,1.816141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215490,RGI60-19.02748,-53.9860,-37.7325,29.9,310,-999,510,0.042,315,255,5.502906e+05,0.042500,19,0,12.948013,85.513069,72.565056,6.604339
215491,RGI60-19.02749,-54.8310,-36.1361,23.6,330,-999,830,0.567,200,1130,1.300672e+07,0.568125,19,0,22.894110,94.770271,71.876162,4.139505
215492,RGI60-19.02750,-54.1884,-37.3018,16.8,10,-999,1110,4.118,308,4329,2.506893e+08,4.118125,19,0,60.874612,88.712219,27.837608,1.457294
215493,RGI60-19.02751,-68.8656,-90.4266,0.4,170,-999,270,0.011,122,106,1.068206e+05,0.010625,19,0,10.053704,108.635857,98.582152,10.805555


In [21]:
df

,RGIId,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,Farinotti Volume,Farinotti Area,region,cluster,Farinotti Thickness,Edasi Thickness,VE - VF,VE / VF
0,RGI60-01.00001,63.6890,-146.8230,42.0,1936,2385,2725,0.360,346,839,7.638771e+06,0.360625,01,0,21.182034,74.723167,53.541133,3.527667
1,RGI60-01.00002,63.4040,-146.6680,16.0,1713,2005,2144,0.558,162,1197,1.697646e+07,0.556250,01,0,30.519479,87.599998,57.080520,2.870298
2,RGI60-01.00003,63.3760,-146.0800,18.0,1609,1868,2182,1.685,175,2106,5.969346e+07,1.680625,01,0,35.518605,85.036270,49.517665,2.394133
3,RGI60-01.00004,63.3810,-146.1200,19.0,1273,1944,2317,3.681,195,4175,1.952248e+08,3.678125,01,0,53.077261,86.705986,33.628725,1.633581
4,RGI60-01.00005,63.5510,-147.0570,16.0,1494,1914,2317,2.573,181,2981,1.221541e+08,2.570625,01,0,47.519206,86.301582,38.782376,1.816141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215490,RGI60-19.02748,-53.9860,-37.7325,29.9,310,-999,510,0.042,315,255,5.502906e+05,0.042500,19,0,12.948013,85.513069,72.565056,6.604339
215491,RGI60-19.02749,-54.8310,-36.1361,23.6,330,-999,830,0.567,200,1130,1.300672e+07,0.568125,19,0,22.894110,94.770271,71.876162,4.139505
215492,RGI60-19.02750,-54.1884,-37.3018,16.8,10,-999,1110,4.118,308,4329,2.506893e+08,4.118125,19,0,60.874612,88.712219,27.837608,1.457294
215493,RGI60-19.02751,-68.8656,-90.4266,0.4,170,-999,270,0.011,122,106,1.068206e+05,0.010625,19,0,10.053704,108.635857,98.582152,10.805555


In [ ]:
arch = '10-5'
top_learning_rate = 0.01
epochs = 100
dropout = 1

fig = plt.figure(figsize=(25, 25))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(
    'Volume estimates using layer architecture ' + str(arch) +
    ', learning rate: ' + str(top_learning_rate) + ', epochs: ' + str(epochs)
             , fontsize=24, y=0.98
            )
fig.patch.set_facecolor('w')

sns.kdeplot(
    x = df['Farinotti Thickness'],
    y = df['Edasi Thickness'],
    fill = True
#         marker = '.',
#     c = df['cluster'],
#     cmap = 'viridis',
#             alpha = 0.5
)
ax = plt.gca()
ax.set_xlim([0,(df['Farinotti Thickness'].max())])
ax.set_ylim([0,(df['Farinotti Thickness'].max())])
plt.plot(
    (0,(df['Farinotti Thicknes'])),
    (0,(df['Farinotti Thicknes'])),
    '-',
    c = 'orange'

)
     


In [ ]:
fig = plt.figure(figsize=(25, 25))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(
    'Volume estimates using layer architecture ' + str(arch) +
    ', learning rate: ' + str(top_learning_rate) + ', epochs: ' + str(epochs)
             , fontsize=24, y=0.98
            )
fig.patch.set_facecolor('w')

for n, cluster_number in enumerate(range(0,10,1)):
    ax = plt.subplot(5, 2, n + 1)
    plt.scatter(
        x = df['Farinotti Thickness'][df['cluster'] == cluster_number],
        y = df['Edasi Thickness'][df['cluster'] == cluster_number],
        fill = True
#         marker = '.',
    #     c = df['cluster'],
    #     cmap = 'viridis',
    #             alpha = 0.5
    )
    plt.plot(
        (0,(df['Farinotti Thickness'][df['cluster'] == cluster_number].max())),
        (0,(df['Farinotti Thickness'][df['cluster'] == cluster_number].max())),
        '-',
        c = 'orange'

    )
    ax = plt.gca()
    ax.set_xlim([0,(df['Farinotti Thickness'][df['cluster'] == cluster_number].max())])
    ax.set_ylim([0,(df['Farinotti Thickness'][df['cluster'] == cluster_number].max())])

    # plt.colorbar()
    plt.xlabel('Farinotti Thickness (m)')
    plt.ylabel('Edasi Thickness (m)')
    ax.set_title('cluster ' +str(cluster_number))
    plt.yscale('log')
    plt.xscale('log')

In [57]:
df

,RGIId,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,Farinotti Volume,Farinotti Area,region,cluster,Farinotti Thickness,Edasi Thickness,VE - VF,VE / VF
0,RGI60-01.00001,63.6890,-146.8230,42.0,1936,2385,2725,0.360,346,839,7.638771e+06,0.360625,01,9,21.182034,74.723167,53.541133,3.527667
1,RGI60-01.00002,63.4040,-146.6680,16.0,1713,2005,2144,0.558,162,1197,1.697646e+07,0.556250,01,9,30.519479,87.599998,57.080520,2.870298
2,RGI60-01.00003,63.3760,-146.0800,18.0,1609,1868,2182,1.685,175,2106,5.969346e+07,1.680625,01,9,35.518605,85.036270,49.517665,2.394133
3,RGI60-01.00004,63.3810,-146.1200,19.0,1273,1944,2317,3.681,195,4175,1.952248e+08,3.678125,01,1,53.077261,86.705986,33.628725,1.633581
4,RGI60-01.00005,63.5510,-147.0570,16.0,1494,1914,2317,2.573,181,2981,1.221541e+08,2.570625,01,1,47.519206,86.301590,38.782384,1.816141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215490,RGI60-19.02748,-53.9860,-37.7325,29.9,310,-999,510,0.042,315,255,5.502906e+05,0.042500,19,5,12.948013,85.513069,72.565056,6.604339
215491,RGI60-19.02749,-54.8310,-36.1361,23.6,330,-999,830,0.567,200,1130,1.300672e+07,0.568125,19,5,22.894110,94.770271,71.876162,4.139505
215492,RGI60-19.02750,-54.1884,-37.3018,16.8,10,-999,1110,4.118,308,4329,2.506893e+08,4.118125,19,1,60.874612,88.712219,27.837608,1.457294
215493,RGI60-19.02751,-68.8656,-90.4266,0.4,170,-999,270,0.011,122,106,1.068206e+05,0.010625,19,5,10.053704,108.635857,98.582152,10.805555


In [58]:
residuals = pd.DataFrame()

for region_number in tqdm(range(1,20,1)):
    if len(str(region_number)) == 1:
        N = 1
        region_number = str(region_number).zfill(N + len(str(region_number)))
    else:
        str(region_number) == str(region_number)
        
    for cluster in df['cluster'][df['region'] == str(region_number)].unique():
        dft = df[
            (df['cluster'] == cluster) & 
            (df['region'] == str(region_number))
        ]
        
        residuals = residuals.append(pd.Series(cluster), ignore_index = True)
        residuals.loc[residuals.index[-1], 'region'] = region_number
        residuals.loc[residuals.index[-1], 'cluster'] = cluster
        residuals.loc[residuals.index[-1], 'count'] = len(dft)
        
        residuals.loc[
            residuals.index[-1], 'VE - VF mean'
        ] = dft['VE - VF'].mean()        
        
        residuals.loc[
            residuals.index[-1], 'VE - VF median'
        ] = dft['VE - VF'].median()  
        
        residuals.loc[
            residuals.index[-1], 'VE / VF mean'
        ] = dft['VE / VF'].mean()  
        
        residuals.loc[
            residuals.index[-1], 'VE / VF median'
        ] = dft['VE / VF'].median()  

        residuals.loc[
            residuals.index[-1], 'Area'
        ] = dft['Area'].mean()  
        
        residuals.loc[
            residuals.index[-1], 'Length'
        ] = dft['Lmax'].mean()          
        
        residuals.loc[       
            residuals.index[-1], 'Slope'
        ] = dft['Slope'].mean()          
        
        residuals.loc[        
            residuals.index[-1], 'Zmin'
        ] = dft['Zmin'].mean()          
        
        residuals.loc[
            residuals.index[-1], 'Zmed'
        ] = dft['Zmed'].mean()          
        
        residuals.loc[
            residuals.index[-1], 'Zmax'
        ] = dft['Zmax'].mean()          
        residuals.loc[
            residuals.index[-1], 'Farinotti Thickness'
        ] = dft['Farinotti Thickness'].mean()  
        residuals.loc[
            residuals.index[-1], 'Edasi Thickness'
        ] = dft['Edasi Thickness'].mean()         
        
        
        
residuals['count'] = residuals['count'].astype(int)

residuals['cluster'] = residuals['cluster'].astype(int)
residuals = residuals.drop(0, axis = 1)
cluster_residuals = pd.DataFrame()

# print(residuals)
for cluster in range(0,10,1):
#     print(cluster)
    dfr = residuals[residuals['cluster'] == cluster]
    cluster_residuals = pd.concat([cluster_residuals, pd.Series(cluster, name = 'cluster')])
    cluster_residuals = cluster_residuals.reset_index()
    cluster_residuals = cluster_residuals.drop('index', axis = 1)
    cluster_residuals.loc[
        cluster_residuals.index[-1], 'Farinotti Thickness'
    ] = dfr['Farinotti Thickness'].mean()
    cluster_residuals.loc[
        cluster_residuals.index[-1], 'Edasi Thickness'
    ] = dfr['Edasi Thickness'].mean() 
    
    cluster_residuals.loc[
        cluster_residuals.index[-1], 'VE - VF'
    ] = dfr['VE - VF mean'].mean()        

    cluster_residuals.loc[
        cluster_residuals.index[-1], 'VE / VF'
    ] = dfr['VE / VF mean'].mean()  
    
    cluster_residuals.loc[
        cluster_residuals.index[-1],'Area'
    ] = dfr['Area'].mean()
    
    cluster_residuals.loc[
        cluster_residuals.index[-1],'Length'
    ] = dfr['Length'].mean()
    
    cluster_residuals.loc[
        cluster_residuals.index[-1],'Slope'
    ] = dfr['Slope'].mean()
    
    cluster_residuals.loc[
        cluster_residuals.index[-1],'Zmin'
    ] = dfr['Zmin'].mean()
    
    cluster_residuals.loc[
        cluster_residuals.index[-1],'Zmed'
    ] = dfr['Zmed'].mean()
    
    cluster_residuals.loc[
        cluster_residuals.index[-1],'Zmax'
    ] = dfr['Zmax'].mean()
    
cluster_residuals = cluster_residuals.rename(columns = {
    0:'cluster'
})   
cluster_residuals['cluster'] = cluster_residuals['cluster'].astype(int)
cluster_residuals[
    'Area / Length (km)'] = ((
    cluster_residuals['Area'] * 1e6
        ) / cluster_residuals['Length']) / 1e3
cluster_residuals[
    'Zmax - Zmin (km)'] = ((
    cluster_residuals['Zmax'] - cluster_residuals['Zmin']
)) / 1e3
cluster_residuals.sort_values('VE / VF')

100%|██████████| 19/19 [00:02<00:00,  7.73it/s]


,cluster,Area,Edasi Thickness,Farinotti Thickness,Length,Slope,VE - VF,VE / VF,Zmax,Zmed,Zmin,Area / Length (km),Zmax - Zmin (km)
3,3,98.653872,113.740936,176.288302,18715.864320,12.767523,-62.547367,0.680420,3108.106192,2101.600794,1184.335529,5.271136,1.923771
6,6,27.375695,82.840161,115.549153,9438.984685,15.510054,-32.688484,1.042477,2975.627433,2222.255632,1541.962717,2.900280,1.433665
2,2,398.665510,312.396840,261.491000,36077.060763,10.196679,50.905848,1.172290,3025.474622,1931.524739,1016.771143,11.050388,2.008703
8,8,4.374072,91.959778,56.025251,4100.242462,21.474247,35.934528,1.764319,5035.964533,4386.005527,3753.130028,1.066784,1.282835
1,1,5.675309,87.314509,69.686208,3706.386360,17.264394,17.636210,1.934024,2031.773721,1535.156597,1167.558196,1.531224,0.864216
7,7,1051.603817,793.238023,345.491821,63070.123767,9.117636,447.746195,2.290085,2948.236801,1676.646083,669.092730,16.673565,2.279144
0,0,0.415261,88.333699,21.991077,849.258649,28.393489,66.342622,4.497694,5491.973492,5262.528285,5019.283767,0.488969,0.472690
9,9,0.336869,86.552406,23.244877,660.146959,29.336131,63.307528,4.603548,2349.225114,1973.018651,1931.076577,0.510295,0.418149
4,4,0.302848,87.828420,21.298912,651.741040,28.835295,66.529507,5.295526,3847.584688,3352.619792,3505.625433,0.464675,0.341959
5,5,0.952216,87.944149,31.337918,861.975485,20.630862,56.605768,5.469498,1058.038904,820.295735,773.403212,1.104691,0.284636


In [ ]:
dfr = df[df['cluster'] == 1]
dfr['Lmax'].min()

In [ ]:
dfy = df[df['cluster'] == 0]
dfy['Lmax'].min()

In [ ]:
dft = df[df['region'] == '05']
for cluster in dft['cluster'].unique():
    dfy = dft[dft['cluster'] == cluster]
    print('cluster ' + str(cluster) + ' has '+ str(len(dfy)) + ' glaciers')

In [ ]:
residuals = pd.DataFrame()
dfy = df[df['region'] == '01']
for cluster_number in range(0,10,1):
    dft = df[df['cluster'] == cluster_number]
    cluster = pd.Series(
        int(df['cluster'][df['cluster'] == cluster_number].iloc[-1]),
        name = 'cluster'
    )
    residuals = pd.concat([residuals, cluster], axis = 0)
    residuals = residuals.drop(0, axis = 1)
    residuals = residuals.reset_index()
    residuals = residuals.drop('index', axis = 1)
    residuals.loc[residuals.index[-1], 'cluster'] = cluster_number
    residuals.loc[residuals.index[-1], 'VE - VF mean'] = dft['VE - VF'].mean()
    residuals.loc[residuals.index[-1], 'VE - VF median'] = dft['VE - VF'].median()
    residuals.loc[residuals.index[-1], 'VE / VF mean'] = dft['VE / VF'].mean()
    residuals.loc[residuals.index[-1], 'VE / VF median'] = dft['VE / VF'].median()
    residuals.loc[residuals.index[-1], 'count'] = len(dft)
residuals['cluster'] = residuals['cluster'].astype(int)
residuals['count'] = residuals['count'].astype(int)

residuals

In [ ]:
dft

In [ ]:
sum(residuals['count'])

In [ ]:
for cluster_number in range(0,10,1):
    dft = df[df['cluster'] == cluster_number]
    cluster = pd.Series(
        int(df['cluster'][df['cluster'] == cluster_number].iloc[-1]),
        name = 'cluster'
    )
    residuals = pd.concat([residuals, cluster], axis = 0)
    residuals = residuals.drop(0, axis = 1)
    residuals = residuals.reset_index()
    residuals = residuals.drop('index', axis = 1)
    residuals.loc[residuals.index[-1], 'cluster'] = cluster_number
    residuals.loc[residuals.index[-1], 'VE - VF mean'] = dft['VE - VF'].mean()
    residuals.loc[residuals.index[-1], 'VE - VF median'] = dft['VE - VF'].median()
    residuals.loc[residuals.index[-1], 'VE / VF mean'] = dft['VE / VF'].mean()
    residuals.loc[residuals.index[-1], 'VE / VF median'] = dft['VE / VF'].median()
    residuals.loc[residuals.index[-1], 'count'] = len(dft)
residuals['cluster'] = residuals['cluster'].astype(int)
residuals['count'] = residuals['count'].astype(int)

In [ ]:
df['VE - VF'].mean()

In [ ]:
for i in range(0,10):
    print('cluster number ' + str(i))
    dft = df[df['cluster'] == i]
    print(list(dft['region'].unique()))
    print('')

In [ ]:
cccombo_breaker()

In [ ]:

for i in range(0,10):
    
    dft = df[df['cluster'] == i]
    print('cluster ' + str(i) + ' Area maximum = ' + str(dft['Area'].max()))
    print('cluster ' + str(i) + ' Area minimum = ' + str(dft['Area'].min()))
    print('cluster ' + str(i) + ' Lmax maximum = ' + str(dft['Lmax'].max()))
    print('cluster ' + str(i) + ' Lmax minimum = ' + str(dft['Lmax'].min()))
    print('cluster ' + str(i) + ' Slope maximum = ' + str(dft['Slope'].max()))
    print('cluster ' + str(i) + ' Slope minimum = ' + str(dft['Slope'].min()))
    print('cluster ' + str(i) + ' Zmin maximum = ' + str(dft['Zmin'].max()))
    print('cluster ' + str(i) + ' Zmin minimum = ' + str(dft['Zmin'].min()))
    print('cluster ' + str(i) + ' Zmed maximum = ' + str(dft['Zmed'].max()))
    print('cluster ' + str(i) + ' Zmed minimum = ' + str(dft['Zmed'].min()))
    print('cluster ' + str(i) + ' Zmax maximum = ' + str(dft['Zmax'].max()))
    print('cluster ' + str(i) + ' Zmax minimum = ' + str(dft['Zmax'].min()))
    print('')

In [ ]:
df

In [ ]:

# print(RGI['RGIId'])

pth_1 = '/home/simonhans/data/prethicktor/RGI/outlines/'
# rootdir = '~'
for region_number in range(1,20,1):
            
    if len(str(region_number)) == 1:
        N = 1
        region_number = str(region_number).zfill(N + len(str(region_number)))
    else:
        str(region_number) == str(region_number)
        
    region_folder = pth_1 + 'RGI60-' + str(region_number) + '/'
    for file in tqdm(os.listdir(region_folder)):
        im = Image.open(region_folder + file)
        imarray = np.array(im)
        df = pd.DataFrame(imarray)
        df = df.replace(0.0, np.nan)
        print(df)
        mean_glacier_thickness = df.mean().mean()
        print(mean_glacier_thickness)
        RGI['Farinotti Mean Thickness'].loc[RGI['RGIId'] == file[:14]] = mean_glacier_thickness
        break
    break

In [ ]:
#### KMeans elbow plot for RGI at fegion scale 

dft = df[[
    'CenLat',
    'CenLon',
    'Slope',
    'Zmin',
    'Zmed',
    'Zmax',
    'Area',
    'Aspect',
    'Lmax',
#     'Farinotti Mean Thickness',
#     'Edasi Mean Thickness'
]]
fig, ax = plt.subplots(1, 1, figsize = (20,20))
plt.subplots_adjust(hspace=0.5)
fig.patch.set_facecolor('w')
# ELBOW PLOT START

x = dft.iloc[:,[0,1,2,3,4,5,6,7,8]]
print(x)
distortions = []
K = range(1,10)
for k in tqdm(K):
    kmeanModel = KMeans(n_clusters = k)
    kmeanModel.fit(x)
    distortions.append(kmeanModel.inertia_)
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.suptitle(
    'The Elbow Method showing the optimal clusters', 
    fontsize=18, y=0.95)

# ELBOW PLOT END

plt.show()    